# SkinVisionNet
Deep Learning Techniques for Accurate Pigmented Skin Lesion Classification



## Dataset retrieval

_Link Dataset:_ https://www.kaggle.com/datasets/wanderdust/skin-lesion-analysis-toward-melanoma-detection/data


In [1]:
import kagglehub
import outputs

path = kagglehub.dataset_download("wanderdust/skin-lesion-analysis-toward-melanoma-detection")

print("Path to dataset files:", path)


C:\Users\colam\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Resuming download from 333447168 bytes (11810070343 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/wanderdust/skin-lesion-analysis-toward-melanoma-detection?dataset_version_number=1 (333447168/12143517511) bytes left.


100%|██████████| 11.3G/11.3G [20:39<00:00, 9.53MB/s] 

Extracting files...


Path to dataset files: C:\Users\colam\.cache\kagglehub\datasets\wanderdust\skin-lesion-analysis-toward-melanoma-detection\versions\1


## Using Dataset from local path



In [10]:
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms

data_dir = "skin-lesions"

train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "valid")  #
test_dir = os.path.join(data_dir, "test")

# Normalizzazione standard per modelli preaddestrati su ImageNet
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

# Data augmentation per il training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

# Transform per validation e test
test_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

# Dataset aggiornati
train_dataset = ImageFolder(root=train_dir, transform=train_transform)
val_dataset = ImageFolder(root=val_dir, transform=test_val_transform)
test_dataset = ImageFolder(root=test_dir, transform=test_val_transform)


## Data Loader setup

In [19]:
from torch.utils.data import DataLoader

batch_size = 32  # Puoi modificare questo valore in base alle risorse disponibili

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


## Fine tuning on Swin Transformer

In [38]:
import timm
import torch.nn as nn

# Determina il numero di classi per la classificazione
num_classes = len(train_dataset.classes)

# Carica del modello Swin Transformer pre-addestrato
model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=num_classes)


In [39]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [48]:
print(torch.cuda.is_available())


False


In [46]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss, correct = 0.0, 0

    for inputs, labels in tqdm(dataloader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        # 🔍 Debug temporaneo
        print("Outputs shape:", outputs.shape)
        print("Labels shape:", labels.shape)
        break  # fermiamo al primo batch solo per analizzare

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(dim=1)
        correct += int((preds == labels).sum())

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct / len(dataloader.dataset)
    return epoch_loss, epoch_acc


def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss, correct = 0.0, 0

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [41]:
**best_val_acc = 0.0
n_epochs = 10
model_path = "best_swin_model.pt"

for epoch in range(n_epochs):
    print(f"\nEpoch {epoch+1}/{n_epochs}")

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    scheduler.step()

    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f"📦 Model saved: {model_path}")





Epoch 1/10


Training:   0%|          | 0/63 [00:00<?, ?it/s]

Outputs shape: torch.Size([32, 3])
Labels shape: torch.Size([32])


Validation: 100%|██████████| 5/5 [00:26<00:00,  5.21s/it]
C:\Users\colam\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\optim\lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Train Loss: 0.0000, Accuracy: 0.0000
Val   Loss: 1.0827, Accuracy: 0.3800
📦 Model saved: best_swin_model.pt

Epoch 2/10


...

Train Loss: 0.0000, Accuracy: 0.0000
Val   Loss: 1.0827, Accuracy: 0.3800

Epoch 10/10


Training:   0%|          | 0/63 [00:00<?, ?it/s]

Outputs shape: torch.Size([32, 3])
Labels shape: torch.Size([32])


Validation: 100%|██████████| 5/5 [00:37<00:00,  7.42s/it]

Train Loss: 0.0000, Accuracy: 0.0000
Val   Loss: 1.0827, Accuracy: 0.3800


In [50]:
# Carica il modello migliore
model.load_state_dict(torch.load(model_path))
model.eval()

test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"\n🎯 Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")


Validation: 100%|██████████| 19/19 [01:31<00:00,  4.79s/it]


🎯 Test Accuracy: 0.3017 | Loss: 1.1340
